In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '3'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
num_ts = 862

In [6]:
# Download the dataset
download_traffic_dataset()

# Clean the dataset
traffic_data = clean_traffic(num_ts=num_ts)
traffic_data.shape

Files not exist
/home/qula0496/quan/Nonstationary_Transformers/utils/data_utils.py
data loaded..


(17544, 862)

In [7]:
traffic_data.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860,MT_861,MT_862
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,0.0375,...,0.0051,0.0051,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121
2015-01-01 01:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,0.0381,...,0.0036,0.0036,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136
2015-01-01 02:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,0.0309,...,0.0030,0.0030,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107
2015-01-01 03:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,0.0199,...,0.0033,0.0033,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071
2015-01-01 04:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,0.0226,...,0.0049,0.0049,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039


In [8]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# traffic_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/traffic/traffic.csv', index_label='date')

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(traffic_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/traffic/'
args.data_path = 'traffic.csv'
args.model_id = 'traffic_96_96'
args.data_path

'traffic.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from datetime import datetime
datetime.strptime('2015-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

datetime.datetime(2015, 1, 1, 0, 0)

In [14]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(86, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(86, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

In [15]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Traffic dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Traffic dataset",
    # Hyperparams
    config={
        "dataset": "Traffic860",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 860,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [16]:
# len(server.test_loader[0])

In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 101.34275245666504
Epoch: 2 cost time: 124.1690092086792
Epoch: 3 cost time: 141.78920340538025
Epoch: 4 cost time: 219.18019914627075
Epoch: 5 cost time: 248.70971512794495


Progress:   3%|▎         | 1/30 [15:28<7:28:41, 928.34s/it]

Epoch: 1 cost time: 268.0891737937927
Epoch: 2 cost time: 268.1454873085022
Epoch: 3 cost time: 268.04471945762634
Epoch: 4 cost time: 268.08643436431885
Epoch: 5 cost time: 268.4338369369507


Progress:   7%|▋         | 2/30 [39:25<9:32:47, 1227.42s/it]

...

Progress:  93%|█████████▎| 28/30 [10:43:20<43:41, 1310.64s/it]  

Epoch: 1 cost time: 243.3594160079956
Epoch: 2 cost time: 241.8831729888916
Epoch: 3 cost time: 241.30721855163574
Epoch: 4 cost time: 242.5002682209015
Epoch: 5 cost time: 242.30870532989502


Progress:  97%|█████████▋| 29/30 [11:04:59<21:47, 1307.19s/it]

Epoch: 1 cost time: 243.30097317695618
Epoch: 2 cost time: 243.493230342865
Epoch: 3 cost time: 244.42128682136536
Epoch: 4 cost time: 243.1486611366272
Epoch: 5 cost time: 243.40658473968506


Progress: 100%|██████████| 30/30 [11:26:45<00:00, 1373.53s/it]


In [18]:
total_mae

[0.34313673,
 0.38036826,
 0.3625005,
 0.3530789,
 0.34848684,
 0.27234304,
 0.34942037,
 0.39622217,
 0.4255446,
 0.49193102]